In [13]:
%pip install numpy pandas matplotlib seaborn opencv-python

Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt 

# OpenCV Implementations for Space Resection

In [ ]:
def get_rotation_matrix_from_euler(roll, pitch, yaw):
    """
    by inserting roll, pitch and yaw angles in degrees, this function returns the rotation matrix
    
    :param roll: rotation around x-axis in degrees
    :param pitch: rotation around y-axis in degrees
    :param yaw: rotation around z-axis in degrees
    :return: rotation matrix
    """
    roll = np.radians(roll)
    pitch = np.radians(pitch)
    yaw = np.radians(yaw)
    # Rotation matrix for roll (rotation around x-axis)
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    # Rotation matrix for pitch (rotation around y-axis)
    R_y = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    # Rotation matrix for yaw (rotation around z-axis)
    R_z = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    # Combined rotation matrix: R = R_z * R_y * R_x
    R = R_z @ R_y @ R_x
    return R

In [16]:
def get_euler_from_rotation_matrix(R):
    """
    by inserting a rotation matrix, this function returns the roll, pitch and yaw angles in degrees
    :param R: rotation matrix
    :return: roll, pitch and yaw angles in degrees
    """
    # Ensure R is a 3x3 matrix
    assert R.shape == (3, 3), "Input must be a 3x3 rotation matrix"
 
    # Extract Euler angles from rotation matrix
    pitch = np.arctan2(-R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2))
    roll = np.arctan2(R[2, 1], R[2, 2])
    yaw = np.arctan2(R[1, 0], R[0, 0])
 
    # Convert radians to degrees
    roll = np.degrees(roll)
    pitch = np.degrees(pitch)
    yaw = np.degrees(yaw)
 
    return roll, pitch, yaw

In [17]:
with open('object_points_m.txt', 'r') as f:
    object_points_m = np.loadtxt(f, delimiter=',', unpack=True)

print(object_points_m)
print('\n',object_points_m.shape)

[[-2.0824  1.1231  9.9714]
 [-1.9727 -0.2535  9.9443]
 [-2.0104  3.156   9.5477]
 [-0.8683 -0.0179 10.8793]
 [-0.8551  1.3556 10.0008]
 [-0.8872  1.2829 10.9052]
 [-1.0292  2.5402 10.9755]
 [-0.9203  3.1915 10.9795]
 [-0.0216  4.1537 10.9914]
 [ 0.      2.0417 10.0002]
 [ 1.0255  3.1893 10.9647]
 [ 1.0422  2.537  10.9544]
 [ 0.9469  1.3068 10.8738]
 [ 0.9403 -0.1906 10.8603]
 [ 1.5626  2.4429  9.9858]
 [ 2.5371  3.1341  9.9612]
 [ 2.4938  1.4519  9.9527]
 [ 2.5663 -0.1119  9.9444]]

 (18, 3)


In [18]:
with open('image_points_pix.txt', 'r') as f:
    image_points_pix = np.loadtxt(f, delimiter=',', unpack=True)
print(image_points_pix)
print('\n',image_points_pix.shape)

[[-8.59129087e+02 -2.28735207e+02]
 [-8.30451510e+02 -6.52666689e+02]
 [-7.67814818e+02  3.74031091e+02]
 [-6.39067914e+02 -6.73745711e+02]
 [-5.05946118e+02 -1.66201107e+02]
 [-6.42685565e+02 -2.08626937e+02]
 [-6.96091525e+02  2.44846514e+02]
 [-6.57529889e+02  4.80937614e+02]
 [-3.43323311e+02  8.49787497e+02]
 [-2.40733057e+02  5.11048216e+01]
 [ 3.85555442e+01  5.03652476e+02]
 [ 4.39228801e+01  2.53791262e+02]
 [ 8.07783500e+00 -2.14454024e+02]
 [-3.32591559e-01 -7.78479624e+02]
 [ 2.72722123e+02  1.86508018e+02]
 [ 6.14640710e+02  4.27180900e+02]
 [ 5.93843602e+02 -1.54068146e+02]
 [ 6.14277410e+02 -6.97652945e+02]]

 (18, 2)


In [30]:
# assuming K camera intrinsic matrix and R rotation matrix
K = np.array([
    [800, 0, 320],
    [0, 800, 240],
    [0,   0,   1]
], dtype=np.float32)

roll = 0  # Rotation around x-axis (in degrees)
pitch = 10  # Rotation around y-axis (in degrees)
yaw = 5  # Rotation around z-axis (in degrees)

In [43]:
R_custom = get_rotation_matrix_from_euler(roll, pitch, yaw)
print(f"Rotation matrix:\n{R_custom}\n")
t_custom = np.array([[0], [0], [-1]], dtype=np.float32)  # Translation
# Combine rotation and translation (Extrinsic matrix)
RT_custom = np.hstack((R_custom, t_custom))
print(f"Extrinsic matrix:\n{RT_custom}")

Rotation matrix:
[[ 0.98106026 -0.08715574  0.17298739]
 [ 0.08583165  0.9961947   0.01513444]
 [-0.17364818  0.          0.98480775]]

Extrinsic matrix:
[[ 0.98106026 -0.08715574  0.17298739  0.        ]
 [ 0.08583165  0.9961947   0.01513444  0.        ]
 [-0.17364818  0.          0.98480775 -1.        ]]


In [37]:
# Finding projection matrix by multiplying intrinsic and extrinsic matrices
P_custom = K @ RT_custom
P_custom = P_custom / P_custom[2, 3]  # normalization
print(f"Projection matrix:\n{P_custom}")

Projection matrix:
[[-7.29280793e+02  6.97245942e+01 -4.53528396e+02  3.20000000e+02]
 [-2.69897583e+01 -7.96955758e+02 -2.48461409e+02  2.40000000e+02]
 [ 1.73648178e-01 -0.00000000e+00 -9.84807753e-01  1.00000000e+00]]


In [39]:
# finding euler angles from rotation matrix
r, p, y = get_euler_from_rotation_matrix(R_custom)
print("Roll:", r)
print("Pitch:", p)
print("Yaw:", y)

Roll: 0.0
Pitch: 10.0
Yaw: 5.0


In [ ]:
[success, rvec, tvec] = cv.solvePnP(object_points_m, image_points_pix, K, None)
"""
cv2.solvePnP() (Perspective n-point) algorithm estimates the pose or position in the space using a set of object points, 
their corresponding image projections, the camera intrinsic matrix and the distortion coefficients.

In this case, we don't use distortion coefficients.

The function returns the rotation vector (rvec) and the translation vector (tvec)

Using Rodrigues() algorithm, we can convert the rotation vector to a rotation matrix.
"""
print("Rotation Vector:\n", rvec)
print("\nTranslation Vector:\n", tvec)

Rotation Vector:
 [[ 2.88947373]
 [-4.54445868]
 [ 3.19036203]]

Translation Vector:
 [[-2.17427822]
 [-2.81952588]
 [-7.44999422]]


In [44]:
R_est, _ = cv.Rodrigues(rvec)
print("Rotation Matrix with Rodrigues method:\n", R_est)

Rotation Matrix with Rodrigues method:
 [[ 0.99977665  0.01204795  0.01736379]
 [-0.01223821  0.99986578  0.01089281]
 [-0.01723023 -0.01110288  0.9997899 ]]


# Manual implementation

In [46]:
hom_object_points_m = np.hstack((object_points_m, np.ones(shape=(object_points_m.shape[0], 1))))
hom_image_points_pix = np.hstack((image_points_pix, np.ones(shape=(image_points_pix.shape[0], 1))))
print(hom_object_points_m.shape)
print(hom_image_points_pix.shape)

(18, 4)
(18, 3)


In [47]:
# https://cseweb.ucsd.edu/classes/wi07/cse252a/homography_estimation/homography_estimation.pdf

A = np.zeros(shape=(2*object_points_m.shape[0], 12))

for i in range(object_points_m.shape[0]):
    x, y = image_points_pix[i, 0], image_points_pix[i, 1]
    X, Y, Z = object_points_m[i, 0], object_points_m[i, 1], object_points_m[i, 2]

    # coefficients of the collinearity condition equations
    A[2*i] = [-X, -Y, -Z, -1, 0, 0, 0, 0, x*X, x*Y, x*Z, x]
    A[2*i+1] = [0, 0, 0, 0, -X, -Y, -Z, -1, y*X, y*Y, y*Z, y]

print(A.shape)

(36, 12)


In [48]:
from scipy.linalg import svd

U, S, Vh = svd(A)

## Projection Matrix

In [49]:
# approximating projection matrix
P_appr = Vh[-1, :]
P_appr = P_appr.reshape(3, 4)
P_appr = P_appr / P_appr[2, 3]  # normalization
print(P_appr)

[[ 1.39840170e+02  1.58089836e+00 -2.63615643e+01  1.55108314e+02]
 [-9.12728974e-01  1.41950386e+02  2.14996981e+00 -2.88690618e+02]
 [-1.15080241e-02  3.62972440e-04 -5.63994371e-02  1.00000000e+00]]


## Mean squared error of reprojection of image coordinates

### 2D projection of 3D points

In [50]:
# Re-projection of 3D points to 2D points
hom_object_points_pix = hom_object_points_m @ P_appr.T

In [51]:
hom_object_points_pix /= hom_object_points_pix[:, 2].reshape(-1, 1)
hom_object_points_pix = hom_object_points_pix[:, :2]
print(hom_object_points_pix)

[[-8.59716709e+02 -2.29284447e+02]
 [-8.30095698e+02 -6.52929122e+02]
 [-7.67254311e+02  3.73957138e+02]
 [-6.38594537e+02 -6.73686078e+02]
 [-5.06310089e+02 -1.65767266e+02]
 [-6.43047515e+02 -2.08091320e+02]
 [-6.96197878e+02  2.44894309e+02]
 [-6.57249176e+02  4.80978393e+02]
 [-3.43320727e+02  8.50024194e+02]
 [-2.41072453e+02  5.18153845e+01]
 [ 3.91146973e+01  5.03215768e+02]
 [ 4.33444708e+01  2.53399560e+02]
 [ 7.80893471e+00 -2.14391496e+02]
 [ 1.11795880e-02 -7.78701978e+02]
 [ 2.72195142e+02  1.86135133e+02]
 [ 6.15063309e+02  4.27412661e+02]
 [ 5.93830303e+02 -1.54618177e+02]
 [ 6.14436276e+02 -6.97166763e+02]]


In [52]:
# calculating the reprojection mean squared error
v = image_points_pix - hom_object_points_pix
print(v)

[[ 0.58762208  0.54924015]
 [-0.35581227  0.26243325]
 [-0.56050628  0.07395377]
 [-0.47337644 -0.05963275]
 [ 0.36397179 -0.43384123]
 [ 0.36194998 -0.53561733]
 [ 0.10635283 -0.04779437]
 [-0.28071235 -0.04077907]
 [-0.00258417 -0.23669639]
 [ 0.33939525 -0.71056296]
 [-0.55915313  0.43670872]
 [ 0.57840927  0.39170199]
 [ 0.26890028 -0.06252757]
 [-0.34377115  0.22235376]
 [ 0.52698099  0.37288511]
 [-0.42259926 -0.23176097]
 [ 0.0132993   0.55003052]
 [-0.15886605 -0.48618146]]


In [53]:
Vx = v[:, 0]
Vy = v[:, 1]
V_er = np.sum(Vx**2 + Vy**2)

sigma_0 = np.sqrt(V_er / 25)
print("Reprojection Error:", sigma_0)

Reprojection Error: 0.46247376635448867


# Extracting Camera Matrix K and Rotation Matrix with Translation Vector, using Matlab function `decomposecamera(P)`

In [54]:
print("Current Projection Matrix:\n", P_appr)
print(P_appr.shape)

Current Projection Matrix:
 [[ 1.39840170e+02  1.58089836e+00 -2.63615643e+01  1.55108314e+02]
 [-9.12728974e-01  1.41950386e+02  2.14996981e+00 -2.88690618e+02]
 [-1.15080241e-02  3.62972440e-04 -5.63994371e-02  1.00000000e+00]]
(3, 4)


In [ ]:
def rq3(A):
    """https://www.peterkovesi.com/matlabfns/Projective/rq3.m"""
    if not np.all(np.array(A.shape) == [3, 3]):
        raise ValueError('A must be 3x3')

    eps = 1e-10

    A[2, 2] += eps
    c = -A[2, 2] / np.sqrt(A[2, 2]**2 + A[2, 1]**2)
    s = A[2, 1] / np.sqrt(A[2, 2]**2 + A[2, 1]**2)
    Qx = np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    R = np.dot(A, Qx)

    R[2, 2] += eps
    c = R[2, 2] / np.sqrt(R[2, 2]**2 + R[2, 0]**2)
    s = R[2, 0] / np.sqrt(R[2, 2]**2 + R[2, 0]**2)
    Qy = np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
    R = np.dot(R, Qy)

    R[1, 1] += eps
    c = -R[1, 1] / np.sqrt(R[1, 1]**2 + R[1, 0]**2)
    s = R[1, 0] / np.sqrt(R[1, 1]**2 + R[1, 0]**2)    
    Qz = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
    R = np.dot(R, Qz)

    Q = Qz.T @ Qy.T @ Qx.T

    # Adjust R and Q so that the diagonal elements of R are +ve
    for n in range(3):
        if R[n, n] < 0:
            R[:, n] = -R[:, n]
            Q[n, :] = -Q[n, :]

    return R, Q

In [ ]:
def decomposecamera(P):
    """https://www.peterkovesi.com/matlabfns/Projective/decomposecamera.m"""
    
    p1 = P[:, 0]
    p2 = P[:, 1]
    p3 = P[:, 2]

    M = np.array([p1, p2, p3])

    # Perform RQ decomposition of M matrix
    K, Rc_w = rq3(M)

    # Check that R is right handed, if not give warning
    if np.dot(np.cross(Rc_w[:, 0], Rc_w[:, 1]), Rc_w[:, 2]) < 0:
        print('Warning: Note that rotation matrix is left handed')
    
    return K, Rc_w

In [57]:
K, Rc_w = decomposecamera(P_appr)
K /= K[2, 2]
print("\nNormalized Camera Matrix K:\n", K)
print("\nRotation Matrix R:\n", Rc_w)


Normalized Camera Matrix K:
 [[ 1.17382366e-02  3.95379238e-01 -5.27242820e+00]
 [ 2.06616859e-18  5.35401454e+00  3.76686727e-01]
 [-3.08842219e-19 -8.90077925e-18  1.00000000e+00]]

Rotation Matrix R:
 [[ 2.13771911e-03 -2.12506989e-05 -9.99997715e-01]
 [ 8.12867695e-02  9.96690743e-01  1.52588254e-04]
 [-9.96688462e-01  8.12869100e-02 -2.13237225e-03]]


In [58]:
r_, p_, y_ = get_euler_from_rotation_matrix(Rc_w)
print("\nRoll:", r_)
print("Pitch:", p_)
print("Yaw:", y_)


Roll: 91.50267634318419
Pitch: 85.33585126608806
Yaw: 88.49355487276647


# Translation Vector extraction

In [59]:
# we can extract the translation vector directly from the projection matrix
# as it is the last column of the projection matrix without the scale factor
# but first we have to normalize the projection matrix

P_appr_norm = P_appr / P_appr[2, 3]
print("\nNormalized Projection Matrix:\n", P_appr_norm)


Normalized Projection Matrix:
 [[ 1.39840170e+02  1.58089836e+00 -2.63615643e+01  1.55108314e+02]
 [-9.12728974e-01  1.41950386e+02  2.14996981e+00 -2.88690618e+02]
 [-1.15080241e-02  3.62972440e-04 -5.63994371e-02  1.00000000e+00]]


In [60]:
T_appr = P_appr_norm[:2, 3]
print("\nTranslation Vector:\n", T_appr)


Translation Vector:
 [ 155.10831372 -288.69061788]
